In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [45]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.118 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Cache-Control': 'max-age=0',
}
url = 'https://bloomscape.com/shop/featured/self-watering-plants/'
webpage = requests.get(url, headers=headers).text
print(webpage[:100])


<!doctype html>
<html lang="en-US">
<head>
	<meta charset="utf-8">
	<meta name="viewport" content="


In [47]:
soup = BeautifulSoup(webpage, 'lxml')
self_watering_plants = soup.find_all('div', class_="product-container")
print('Total items: ', len(self_watering_plants))

Total items:  9


In [48]:
title = []
size = []
price = []
min_price = []
max_price = []
badge = []

for i in self_watering_plants:
  # title
  title_el = i.find('h2', class_='woocommerce-loop-product__title product-info__title').text.strip()
  if title_el:
    title.append(title_el)
  else:
    title.append(None)

  # size
  size_el = i.find('div', class_='product-info__meta').text.strip()
  if size_el:
    size.append(size_el)
  else:
    size.append(None)

  # price
  price_el = i.find('div', class_='product-info__price').text.strip()
  if price_el:
    price.append(price_el)
  else:
    price.append(None)

  # min and max price
  price_cont = i.find('div', class_='product-info__price')
  if price_cont:
    min_max_prices = []
    price_spans = price_cont.find_all('span', class_='woocommerce-Price-amount')
    for span in price_spans:
      clean_text = span.get_text(strip=True).replace('$', '')
      try:
          min_max_prices.append(float(clean_text))
      except ValueError:
          continue  # Skip invalid

    min_p = min_max_prices[0] if len(min_max_prices) > 0 else None
    max_p = min_max_prices[1] if len(min_max_prices) > 1 else min_p

    min_price.append(min_p)
    max_price.append(max_p)

  # badge
  badge_el = i.find('div', class_='product-badge')
  if badge_el:
    badge.append(badge_el.get_text(strip=True))
  else:
    badge.append(None)

In [49]:
data = {'title': title, 'product_size_category': size, 'price_range': price, 'min_price': min_price, 'max_price': max_price, 'badge': badge}
df = pd.DataFrame(data)
display(df)

,title,product_size_category,price_range,min_price,max_price,badge
0,Peace Lily,MD,$69 – $89,69.0,89.0,Most Gifted
1,Neon Prayer Plant,MD,None,NaN,NaN,Bestseller
2,Money Tree Forest,MD,$89 – $109,89.0,109.0,None
3,Parlor Palm,MD,$69 – $89,69.0,89.0,None
4,Ponytail Palm,MD,$79 – $99,79.0,99.0,None
5,Little Fiddle Leaf Fig,MD,$79 – $99,79.0,99.0,None
6,Baltic Blue Pothos,MD,$79 – $99,79.0,99.0,None
7,Monstera Adansonii,MD,$79 – $99,79.0,99.0,None
8,Money Tree Braid,MD,$69 – $89,69.0,89.0,None
